# Local Peristence Demo
This notebook demonstrates how to configure Chroma to persist to disk, then load it back in. 

In [2]:
%pip install chromadb==0.3.26

  Using cached chromadb-0.3.26-py3-none-any.whl (123 kB)
  Using cached hnswlib-0.7.0-cp311-cp311-win_amd64.whl
  Using cached clickhouse_connect-0.6.9-cp311-cp311-win_amd64.whl (228 kB)
  Attempting uninstall: chromadb
    Found existing installation: chromadb 0.4.7
    Uninstalling chromadb-0.4.7:
      Successfully uninstalled chromadb-0.4.7
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install pyarrow
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

# Create a Pandas DataFrame
data = {
        
        #'column1': [1, 2, 3, 4, 5],
        #'column2': ['a', 'b', 'c', 'd', 'e']

        'embeddings' : [
            [1.1, 2.3, 3.2],
            [4.5, 6.9, 4.4],
            [1.1, 2.3, 3.2],
            [4.5, 6.9, 4.4],
            [1.1, 2.3, 3.2],
            [4.5, 6.9, 4.4],
            [1.1, 2.3, 3.2],
            [4.5, 6.9, 4.4],
        ],
        'metadatas': [
            {"uri": "img1.png", "style": "style1"},
            {"uri": "img2.png", "style": "style2"},
            {"uri": "img3.png", "style": "style1"},
            {"uri": "img4.png", "style": "style1"},
            {"uri": "img5.png", "style": "style1"},
            {"uri": "img6.png", "style": "style1"},
            {"uri": "img7.png", "style": "style1"},
            {"uri": "img8.png", "style": "style1"},
        ],
        'documents': ["doc1", "doc2", "doc3", "doc4", "doc5", "doc6", "doc7", "doc8"],
        'ids': ["id1", "id2", "id3", "id4", "id5", "id6", "id7", "id8"],
        
        
    }
df = pd.DataFrame(data)

# Convert the DataFrame to a PyArrow Table
table = pa.Table.from_pandas(df)

# Define the Parquet file path
file_path = 'parquet\example.parquet'

# Write the PyArrow Table to a Parquet file
pq.write_table(table, file_path)

ModuleNotFoundError: No module named 'pyarrow'

In [4]:
import chromadb

In [26]:
# Create a new Chroma client with persistence enabled. 

settings = chromadb.config.Settings(persist_directory="db\chromadb",chroma_db_impl='duckdb+parquet')

client = chromadb.Client(settings)
# Create a new chroma collection
collection_name = "collection_1"
collection = client.get_or_create_collection(name=collection_name)

print(collection.count())

552


In [4]:
# Add some data to the collection
collection.add(
    embeddings=[
        [1.1, 2.3, 3.2],
        [4.5, 6.9, 4.4],
        [1.1, 2.3, 3.2],
        [4.5, 6.9, 4.4],
        [1.1, 2.3, 3.2],
        [4.5, 6.9, 4.4],
        [1.1, 2.3, 3.2],
        [4.5, 6.9, 4.4],
    ],
    metadatas=[
        {"uri": "img1.png", "style": "style1"},
        {"uri": "img2.png", "style": "style2"},
        {"uri": "img3.png", "style": "style1"},
        {"uri": "img4.png", "style": "style1"},
        {"uri": "img5.png", "style": "style1"},
        {"uri": "img6.png", "style": "style1"},
        {"uri": "img7.png", "style": "style1"},
        {"uri": "img8.png", "style": "style1"},
    ],
    documents=["doc1", "doc2", "doc3", "doc4", "doc5", "doc6", "doc7", "doc8"],
    ids=["id1", "id2", "id3", "id4", "id5", "id6", "id7", "id8"],
)

NameError: name 'collection' is not defined

In [29]:
# Create a new client with the same settings
client = chromadb.PersistentClient(path=persist_directory)

# Load the collection
collection = client.get_collection(collection_name)

In [30]:
# Query the collection
import numpy as np

vector = np.random.rand(1536).astype(np.float32)

results = collection.query(
    query_embeddings=[vectors],
    n_results=1
)

print(results)

ValueError: Expected each embedding in the embeddings to be a list, got [array([[0.35012555, 0.30736738, 0.37419567, ..., 0.17984603, 0.8715208 ,
        0.5717568 ]], dtype=float32)]

In [28]:
collection.get(include=["embeddings", "metadatas", "documents"])

{'ids': [], 'embeddings': [], 'metadatas': [], 'documents': []}

In [7]:
# Clean up
! rm -rf db